* 회귀의 경우 대부분 실제값과 예측값의 오차 평균값에 기반
* 분류의 경우
    * 이진 분류 / 멀티 분류
    * 단순한 정확도만 가지고 판단하면 잘못된 평가 결과에 빠질 수 있다.

## 01. 정확도 (Accuracy)
실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표
정확도 = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수

In [2]:
## 정확도의 잘못된 평가 결과 예시

# 앞의 타이타닉 예제에서 정확도의 결과가 80%대였지만, 여자인 경우 생존 확률이 높았기 때문에,
# 무조건 성별이 여자면 생존, 남자연 사망으로 예측해도 비슷한 수치가 나올 수 있다.

from sklearn.base import BaseEstimator  # 사이킷런은 BaseEstimator를 상속받으면 Customized 형태의 Estimator 생성 가능

class MyDummyClassifier(BaseEstimator):
    # fit() 매서드는 아무것도 학습하지 않음
    def fit(self,X,y=None):
        pass
    # predict() 매서드는 단순이 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self,X):
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i]==1:
                pred[i] =0
            else:
                pred[i] = 1
                
        return pred

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing


# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

# Dummy Classifier를 이용해 학습, 예측, 평가
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도: {0:4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도: 0.787709
